In [27]:
profile_output = "GCA_000013405.1_ASM1340v1_profile_output"
#"GCA_000011185.1_ASM1118v1_profile_output"
lookup = { 0:'A', 1:'C', 2:'G', 3:'T'}
found_matrix = False 
found_start = False
found_end = False
SDs = []
SD = ""
with open(profile_output, "r+") as f:
    for line in f:
        if (line.startswith("% Weight matrix composed")):
            found_matrix = True
        elif (line.startswith("     A")):
            pass
        elif (found_matrix):
            numbers = line.strip().split("\t")
            if (len(numbers) != 6) :
                continue # bug in output
            if (not found_start and float(numbers[5]) >= 0.4): #0.35): #0.4
                found_start = True
                found_end = False
                SD = ""
                #print("found start")
            if (found_start and (float(numbers[5]) < 0.4)): #0.35)): #0.4
                found_end = True
                found_start = False
                #print("found end")
                SDs.append(SD)
            if (found_start and not found_end):
                max_num = float(numbers[0])
                max_i = 0
                for i in range(1,4):
                    if (float(numbers[i]) > max_num):
                        max_num = float(numbers[i])
                        max_i = i
                SD = SD + lookup[max_i]
maxSD = ""
for SD in SDs:
    if(len(SD) > len(maxSD)):
        maxSD = SD
print(SDs)
print(maxSD)

['AGGAG']
AGGAG


In [12]:
# This is an OLD VERSION, see cell below for new one
#### FOR TESTING, remove keep only valid orfs from Tompa search
from subprocess import Popen, PIPE, STDOUT
import subprocess
import csv
import scipy
from os.path import join

# expected perl: v5
# expected input files:
# / *_genomic.fna.gz
# keep_only_valid_orfs.pl, tompa_test installed, 

def run_Tompa_pipeline(file_prefix, input_path, output_path): 

    print(file_prefix)
    rbs_input = output_path + file_prefix + '_RBSidentify_input'
    rbs_output = output_path + file_prefix + '_RBSidentify_output'
    profile_input = output_path + file_prefix + '_profile_input'
    profile_output = output_path + file_prefix + '_profile_output' 
    
    # pre-process to get rid of bad orfs
    cds_file = input_path + file_prefix + "_cds_from_genomic.fna.gz"
    processed_cds_file = output_path + file_prefix + "_cds_processed.fa"
    
    #p = Popen("gunzip" + " -c " + cds_file + " > " + processed_cds_file, shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
    #ret = p.communicate()[0]
    try:
        ret = subprocess.check_output(["perl", "keep_only_valid_orfs.pl", cds_file, processed_cds_file], stderr=subprocess.STDOUT)
        ret = ret.decode("utf-8")
        if (ret != ''):
            print(ret)
            return("") # problem with input
    except subprocess.CalledProcessError as exc:
        print("Problem executing keep_only_valid_orfs for ", cds_file, "; ", exc.returncode, exc.output)
    
    #TODO later make sequence upstream of genes handle multiple chromosomes!!!!    
    full_genome_file = input_path + file_prefix + "_genomic.fna.gz"
    try:
        ret = subprocess.check_output(["perl", "get_sequence_upstream_of_gene.pl", processed_cds_file,\
                                      full_genome_file, rbs_input], stderr=subprocess.STDOUT)
        ret = ret.decode("utf-8")
        if (ret != ''):
            print(ret)
            return("") # problem with input
    except subprocess.CalledProcessError as exc:
        print("Problem executing get_sequence_upstream_of_gene.pl for ", cds_file, "; ", exc.returncode, exc.output)
    
    
    # run RBSidentify
    p = Popen(['tompa_test/RBSidentify/RBSidentify'], stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    stdout = p.communicate(input= (rbs_input + '\n' + rbs_output + '\nn\n7\n1\nn\n').encode())[0]

    
    # process output of RBSidentify 
    p = Popen("tail -n +27 " + rbs_output  + "|  awk '{print $NF,$0}' | sort -nr | cut -f2- -d' ' >"  + profile_input, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    p.wait()

    # run profile
    p = Popen(['tompa_test/profile/profile'], stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    stdout = p.communicate(input=(profile_input + '\n' + rbs_input + '\n'  + profile_output + '\n20\n7\n1\nn\n').encode())[0]

    lookup = { 0:'A', 1:'C', 2:'G', 3:'T'}
    found_matrix = False 
    found_start = False
    found_end = False
    SD = ""
    with open(profile_output, "r+") as f:
        for line in f:
            if (line.startswith("% Weight matrix composed")):
                found_matrix = True
            elif (line.startswith("     A")):
                pass
            elif (found_matrix):
                numbers = line.strip().split()
                if (len(numbers) != 6) :
                    break # bug in output
                if (float(numbers[5]) >= 0.35):
                    found_start = True
                if (found_start and (float(numbers[5]) < 0.35)):
                    found_end = True
                if (found_start and not found_end):
                    max_num = float(numbers[0])
                    max_i = 0
                    for i in range(1,4):
                        if (float(numbers[i]) > max_num):
                            max_num = float(numbers[i])
                            max_i = i
                    SD = SD + lookup[max_i]
                    
    # clean up the files
    #p = Popen("rm " + processed_cds_file, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    #p.wait()
    #p = Popen("rm " + rbs_input, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    #p.wait()
    p = Popen("rm " + rbs_output, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    p.wait()
    p = Popen("rm " + profile_input, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    p.wait()
    #p = Popen("rm " + profile_output, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    #p.wait()
    return(SD)



# expected file
csv_with_file_prefixes = "DiwanDataFilesInfo.csv"
csv_with_tompa_sd_added = 'DiwanDataFiles_WithTompa_no_filter.csv'
input_file_path = "/scratch4/moamin/ncbi_bacteria_db/GbBac_FNA/"
output_file_path = "./"
#file_prefix = "Sulfolobus_acidocaldarius"

count = 0
with open(csv_with_file_prefixes, 'r') as in_file:
    with open(csv_with_tompa_sd_added, 'w') as out_file:
        csvReader = csv.reader(in_file, delimiter=',')
        csvWriter = csv.writer(out_file, delimiter=',')
        for row in csvReader:
            if (row[0] == "name"):
                row.append("tompa_sd")
                csvWriter.writerow(row)
                continue
            count += 1
            if (count > 10):
                break
            file_prefix = row[1]
            #if (file_prefix != "GCA_000946735.1_Vibrio_cholerae_G_298_Guinea"):
             #   continue
            SD = run_Tompa_pipeline(file_prefix, input_file_path, output_file_path)
            if ((len(SD) < 4) or (len(SD) > 6)):
                SD = "------"
            row.append(SD)
            csvWriter.writerow(row)
            
            
# TODO: provide path to the installed tompa algo

GCA_000008885.1_ASM888v1
GCA_000012765.1_ASM1276v1
GCA_000008365.1_ASM836v1
GCA_000007725.1_ASM772v1
GCA_000006625.1_ASM662v1
GCA_000011225.1_ASM1122v1
GCA_000195875.1_ASM19587v1
GCA_000012225.1_ASM1222v1
GCA_000008305.1_ASM830v1
GCA_000007325.1_ASM732v1


In [29]:
from subprocess import Popen, PIPE, STDOUT
import subprocess
import csv
import scipy
from os.path import join
import glob
import time

# expected perl: v5
# expected input files:
# / *_genomic.fna.gz
# keep_only_valid_orfs.pl, tompa_test installed, 

def run_Tompa_pipeline(file_prefix, input_path, output_path): 

    print(file_prefix)
    rbs_input = output_path + file_prefix + '_RBSidentify_input'
    rbs_output = output_path + file_prefix + '_RBSidentify_output'
    profile_input = output_path + file_prefix + '_profile_input'
    profile_output = output_path + file_prefix + '_profile_output' 
    
    # pre-process to get rid of bad orfs
    cds_file = glob.glob(input_path + file_prefix + "*_cds_from_genomic.fna.gz")[0]
    print(cds_file)
    processed_cds_file = output_path + file_prefix + "_cds_processed.fa"
    try:
        #ret = subprocess.check_output(["perl", "keep_only_valid_orfs.pl", cds_file, processed_cds_file], stderr=subprocess.STDOUT)
        ret = subprocess.check_output(["perl", "multi_keep_only_valid_orfs.pl", cds_file, processed_cds_file], stderr=subprocess.STDOUT)
        print("done with multi")
        ret = ret.decode("utf-8")
        if (ret != ''):
            print(ret)
            return("") # problem with input
    except subprocess.CalledProcessError as exc:
        print("Problem executing multi_keep_only_valid_orfs for ", cds_file, "; ", exc.returncode, exc.output)
    
    #TODO later make sequence upstream of genes handle multiple chromosomes!!!!    
    genome_matches = glob.glob(input_path + file_prefix + "*_genomic.fna.gz")
    full_genome_file = ""
    for name in genome_matches:
        if ("from_genomic" not in name):
            full_genome_file = name
    
    print(full_genome_file)
    try:
        ret = subprocess.check_output(["perl", "multi_get_sequence_upstream_of_gene.pl", processed_cds_file,\
                                      full_genome_file, rbs_input], stderr=subprocess.STDOUT)
        print("done with get sequence upstream")
        ret = ret.decode("utf-8")
        if (ret != ''):
            print(ret)
            return("") # problem with input
    except subprocess.CalledProcessError as exc:
        print("Problem executing multi_get_sequence_upstream_of_gene.pl for ", cds_file, "; ", exc.returncode, exc.output)
    
    
    # run RBSidentify
    p = Popen(['tompa_test/RBSidentify/RBSidentify'], stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    stdout = p.communicate(input= (rbs_input + '\n' + rbs_output + '\nn\n7\n1\nn\n').encode())[0]
    
    print("done with RBSidentify")
    # check that the output file was created
    names= glob.glob(rbs_output)
    if (len(names) == 0):
        return("") # problem with input
     #   print("Error, more than 1 file with prefix:", first)
    
    # process output of RBSidentify 
    p = Popen("tail -n +27 " + rbs_output  + "|  awk '{print $NF,$0}' | sort -nr | cut -f2- -d' ' >"  + profile_input, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    p.wait()

    # run profile
    p = Popen(['tompa_test/profile/profile'], stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    stdout = p.communicate(input=(profile_input + '\n' + rbs_input + '\n'  + profile_output + '\n20\n7\n1\nn\n').encode())[0]

    print("done with profile")
    lookup = { 0:'A', 1:'C', 2:'G', 3:'T'}
    found_matrix = False 
    found_start = False
    found_end = False
    
                    
    SDs = []
    SD = ""
    with open(profile_output, "r+") as f:
        last_threshold = 0
        for line in f:
            if (line.startswith("% Weight matrix composed")):
                found_matrix = True
            elif (line.startswith("     A")):
                pass
            elif (found_matrix):
                #print("found_matrix")
                numbers = line.strip().split("\t")
                if (len(numbers) != 6) :
                    break # continue # bug in output
                    #print("if_1")
                if (not found_start and float(numbers[5]) >= 0.4): 
                    found_start = True
                    found_end = False
                    SD = ""
                    #print("if_2")
                if (found_start and (float(numbers[5]) < 0.4) and (last_threshold < 0.4)):
                    found_end = True
                    found_start = False
                    SDs.append(SD[:-1])
                    #print("if_3")
                if (found_start and not found_end):
                    max_num = float(numbers[0])
                    max_i = 0
                    for i in range(1,4):
                        if (float(numbers[i]) > max_num):
                            max_num = float(numbers[i])
                            max_i = i
                    SD = SD + lookup[max_i]
                    #print("if_4")
                last_threshold = float(numbers[5])
    maxSD = ""
    for SD in SDs:
        if(len(SD) > len(maxSD)):
            maxSD = SD              
                    
    # clean up the files
    #p = Popen("rm " + processed_cds_file, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    #p.wait()
    #p = Popen("rm " + rbs_input, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    #p.wait()
    #p = Popen("rm " + rbs_output, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    #p.wait()
    #p = Popen("rm " + profile_input, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    #p.wait()
    #p = Popen("rm " + profile_output, shell=True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
    #p.wait()
    return(maxSD)



# expected file
#csv_with_file_prefixes = ""
#csv_with_tompa_sd_added = ''
csv_with_file_prefixes = "Bruce_July5_orgs.csv"
csv_with_tompa_sd_added = 'Bruce_July5_orgs_withTompa_v4.csv'
#csv_with_file_prefixes = "166_bad_tompa_Bacteria_DataFilesInfo.csv"
#csv_with_tompa_sd_added = '166_bad_tompa_Bacteria_DataFilesInfo_withTompa_v4.csv'
#csv_with_file_prefixes = "Our_NOT_IN_DIWAN_SD_Bacteria_DataFilesInfo.csv"
#csv_with_tompa_sd_added = 'Our_NOT_IN_DIWAN_SD_Bacteria_DataFilesInfo_withTompa_v4.csv'
#csv_with_file_prefixes = "Diwan_MissingArchaea_FilesInfo.csv"
#csv_with_tompa_sd_added = 'Diwan_MissingArchaea_FilesInfo_withTompa_v4.csv'
#csv_with_file_prefixes = "Diwan_MissingBacteria_FilesInfo.csv"
#csv_with_tompa_sd_added = 'Diwan_MissingBacteria_FilesInfo_withTompa_v4.csv'
#csv_with_file_prefixes = "Ruhul_Annotate_153_DataFilesInfo.csv"
#csv_with_tompa_sd_added = 'Ruhul_Annotate_153_DataFiles_WithTompa_v4.csv'
#csv_with_file_prefixes = "DiwanDataFilesInfo.csv"
#csv_with_tompa_sd_added = 'DiwanDataFiles_WithTompa_v4.csv'
#csv_with_file_prefixes = "Our_SD_Bacteria_DataFilesInfo.csv"
#csv_with_tompa_sd_added = 'Our_SD_Bacteria_DataFilesInfo_with_Tompa.csv'
#input_file_path = "/scratch4/moamin/ncbi_archaea_db/GbArchaea_FNA/"
#input_file_path = "/scratch4/moamin/ncbi_bacteria_db/GbBac_FNA/"
input_file_path = "/home/moamin/ncbi_bacteria_db/GbBac_FNA/"
output_file_path = "./"
#file_prefix = "Sulfolobus_acidocaldarius"

count = 0
with open(csv_with_file_prefixes, 'r') as in_file:
    with open(csv_with_tompa_sd_added, 'w') as out_file:
        csvReader = csv.reader(in_file, delimiter=',')
        csvWriter = csv.writer(out_file, delimiter=',')
        for row in csvReader:
            if (row[0] == "name"):
                #row.append("tompa_sd")
                csvWriter.writerow(row)
                continue
            count += 1
            #if (count > 2):
             #   break
            file_prefix = row[1]
            #if (file_prefix != "GCA_000282115.1_Chryseobacterium.strCF314_v1.0"):
             #   continue
            SD = run_Tompa_pipeline(file_prefix, input_file_path, output_file_path)
            print(SD)
            if (len(SD) < 4): 
                SD = "------"
            if (len(SD) > 6):
                SD = SD[:6]
            #print(row[0])
            #print(SD)
            #print(row[3])
            #row.append(SD)
            #if (SD == row[3]):
             #   csvWriter.writerow(row)
            #else:
             #   new_row = row[:3]
            #  new_row.append(SD)
               # csvWriter.writerow(new_row)
            new_row = row[:2]
            new_row.append(SD)
            csvWriter.writerow(new_row)
            
# TODO: provide path to the installed tompa algo

GCA_001273995.1
/home/moamin/ncbi_bacteria_db/GbBac_FNA/GCA_001273995.1_ASM127399v1_cds_from_genomic.fna.gz
done with multi
/home/moamin/ncbi_bacteria_db/GbBac_FNA/GCA_001273995.1_ASM127399v1_genomic.fna.gz
done with get sequence upstream
done with RBSidentify
done with profile
ACACCGAT
GCA_000735325.1
/home/moamin/ncbi_bacteria_db/GbBac_FNA/GCA_000735325.1_ASM73532v1_cds_from_genomic.fna.gz
done with multi
/home/moamin/ncbi_bacteria_db/GbBac_FNA/GCA_000735325.1_ASM73532v1_genomic.fna.gz
done with get sequence upstream
done with RBSidentify
done with profile
AAGGA
GCA_001438645.1
/home/moamin/ncbi_bacteria_db/GbBac_FNA/GCA_001438645.1_ASM143864v1_cds_from_genomic.fna.gz
done with multi
/home/moamin/ncbi_bacteria_db/GbBac_FNA/GCA_001438645.1_ASM143864v1_genomic.fna.gz
done with get sequence upstream
done with RBSidentify
done with profile
TGAGGA
GCA_000268045.1
/home/moamin/ncbi_bacteria_db/GbBac_FNA/GCA_000268045.1_ASM26804v1_cds_from_genomic.fna.gz
done with multi
/home/moamin/ncbi_ba

In [25]:
print("hello")

hello


In [38]:
import scipy.stats
obj = scipy.stats.combine_pvalues([0.042, 0.233])
print(str(format(float(obj[1]), '.3f')))

0.055
